# Spin-Locking amplitude sweep

In [2]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [3]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')   
try:
    poimanagerlogic
except NameError:
    manager.startModule('logic', 'poimaganerlogic')    

In [4]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [5]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV5'
setup['poi'] = poimanagerlogic.active_poi.get_key()
setup['laser_length'] = 4.0e-6
setup["min_counts"] = 10000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

In [10]:
# for single frequency XY8-N:
hh_amp_sweep = OrderedDict()
hh_amp_sweep['mw_amp'] = 0.25
hh_amp_sweep['spinlock_amp_start'] = 0.113
hh_amp_sweep['spinlock_amp_end'] = 0.123
hh_amp_sweep['points'] = 40
hh_amp_sweep['mw_freq'] = 100e6
hh_amp_sweep['tau'] = 60.0 * 1e-6
hh_amp_sweep['measurement_time'] = 240*60
hh_amp_sweep['refocus_interval'] = 3*60
hh_amp_sweep['alternating']=True
hh_amp_sweep['rabi_period']=205.7*1e-9

In [11]:
# generate new HH sequence
sequencegeneratorlogic.delete_sequence('HH_amp_sweep')    
sequencegeneratorlogic.generate_HH_amp_sweep_sequence('HH_amp_sweep', hh_amp_sweep['rabi_period'],
                             hh_amp_sweep['spinlock_amp_start'], hh_amp_sweep['spinlock_amp_end'], 100e6, 0.25, 
                             hh_amp_sweep['tau'], hh_amp_sweep['points'], 'a_ch1', 3.0e-6, 2.0, 0.7e-6, 1.0e-6, 
                             '', 'd_ch2', True)

# sample, upload and load waveform
if 'HH_amp_sweep' not in sequencegeneratorlogic.saved_pulse_sequences:
    sequencegeneratorlogic.log.error('Sequence generation of HH_amp_sweep in notebook timed out.')
pulsedmasterlogic.sample_sequence('HH_amp_sweep', True)

while pulsedmasterlogic.status_dict['sauplo_sequence_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(5)

import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)


In [12]:
refocus_poi(setup['poi'])

186143.0 3.2824039515309844e-08


True

In [13]:
do_HH_amp_sweep_refocus(setup['poi'])

186143.0 3.4190693054656114e-08
186143.0 9.892208885358019e-10
186143.0 6.35048120918569e-09
2D gaussian fit not successfull
186143.0 0.0
186143.0 4.373488913433427e-09
2D gaussian fit not successfull
186143.0 0.0
186143.0 8.943363812885551e-10
186143.0 1.9712105646742023e-09
2D gaussian fit not successfull
186143.0 0.0
186143.0 6.238922099586076e-10
186143.0 4.522063194858033e-09
186143.0 8.15786781618817e-09
186143.0 4.35020297429446e-09
2D gaussian fit not successfull
186143.0 0.0
186143.0 1.010175626050343e-09
186143.0 2.0640129254104428e-09
186143.0 4.158708950426478e-09
186143.0 2.1938028930394585e-09
186143.0 1.1799245623931405e-09
186143.0 8.649579268803123e-10
186143.0 2.244504398089635e-09
186143.0 6.318634368306748e-11
186143.0 2.326770027162101e-09
2D gaussian fit not successfull
186143.0 0.0
186143.0 4.634245247266231e-09
186143.0 8.077168624088106e-10
186143.0 4.343517643552231e-09
186143.0 3.838428837946747e-09
186143.0 3.797543056234466e-09
186143.0 1.0579165871167531e-

Traceback (most recent call last):
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed_measurement_logic.py", line 885, in stop_pulsed_measurement
    self.pulse_generator_off()
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed_measurement_logic.py", line 455, in pulse_generator_off
    self._pulse_generator_device.set_active_channels({'a_ch1': False, 'a_ch2': False}) # to swich on the laser
  File "C:\Users\pi3\Documents\GitHub\qudi\hardware\awg\tektronix_awg5014c.py", line 955, in set_active_channels
    return self.get_active_channels(ch=list(ch))
  File "C:\Users\pi3\Documents\GitHub\qudi\hardware\awg\tektronix_awg5014c.py", line 877, in get_active_channels
    active_ch[channel] = bool(int(self.ask('OUTPUT{0}:STATE?'.format(ana_chan))))
ValueError: invalid literal for int() with base 10: ''
Traceback (most recent call last):
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed_measurement_logic.py", line 885, in stop_pulsed_measurement
    self.pulse_generator_off()
 

True

In [1]:
def refocus_poi(poi=None, max_drift=0.3,min_counts=setup["min_counts"]):
    if poi==None:
        return False
#     pulsedmasterlogic.load_asset_into_channels('Laser_On')
#     while pulsedmasterlogic.status_dict['loading_busy']:
#         time.sleep(0.5)
#     pulsedmasterlogic.toggle_pulse_generator(False)
#     time.sleep(0.5)
    # perform refocus
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.module_state() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.module_state() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

def do_HH_amp_sweep(save_tag='',load_tag=''):
    
    pulsedmasterlogic.do_fit('No Fit')
    mw_array = np.linspace(hh_amp_sweep['spinlock_amp_start'], hh_amp_sweep['spinlock_amp_end'],
                           hh_amp_sweep['points'])
    # set parameters in analysis tab
    if hh_amp_sweep['alternating']:
        pulsedmasterlogic.measurement_sequence_settings_changed(mw_array, 2*hh_amp_sweep['points'], 100e-6, [],
                                                                hh_amp_sweep['alternating'])
    else:
        pulsedmasterlogic.measurement_sequence_settings_changed(mw_array, hh_amp_sweep['points'], 100e-6, [], 
                                                                hh_amp_sweep['alternating'])
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length'], 2*hh_amp_sweep['points'])
    pulsedmasterlogic.analysis_interval_changed(2)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < hh_amp_sweep['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data('s','Spin_Locking_amp_sweep-'+str(hh_amp_sweep['tau'])
                                                +'_'+setup['NV_name'],True, False)
    time.sleep(2)
    return user_terminated

def do_HH_amp_sweep_refocus(poi):
    end_measure=False
    #refocus_poi(poi)
    hh_amp_sweep_total_time = hh_amp_sweep['measurement_time']
    hh_amp_sweep_runtime = 0.0
    hh_amp_sweep['measurement_time'] = hh_amp_sweep['refocus_interval']
    end_measure = do_HH_amp_sweep('hh_amp_sweep','')
    hh_amp_sweep_runtime += hh_amp_sweep['refocus_interval']
    while hh_amp_sweep_total_time > hh_amp_sweep_runtime:
        for i in range(1):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_HH_amp_sweep('hh_amp_sweep', 'hh_amp_sweep')
        hh_amp_sweep_runtime += hh_amp_sweep['refocus_interval']
        if end_measure:
            break
    hh_amp_sweep['measurement_time'] = hh_amp_sweep_total_time
    # save measurement
    pulsedmasterlogic.save_measurement_data('s','Spin_Locking_amp_sweep-'+str(hh_amp_sweep['tau'])
                                                +'_'+setup['NV_name'],True, False)
    write_to_logfile('HH_amp_sweep', str(datetime.datetime.now()),name= setup['NV_name'],
                     runtime=hh_amp_sweep_runtime)
    time.sleep(2)
    return end_measure

In [6]:
mw_freq = 1392.82e+6 # in MHz
mw_power = -15

pulsedmeasurement._pa.ext_control_mw_freq_DoubleSpinBox.setValue(mw_freq-0.1e+9)
pulsedmeasurement._pa.ext_control_mw_power_DoubleSpinBox.setValue(mw_power)
pulsedmeasurement.ext_mw_params_changed()